In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
r_df = pd.read_csv(r"E:\Projects\datasets\IMDB Dataset.csv\IMDB Dataset.csv")

In [4]:
y=r_df['sentiment']
x=r_df['review']

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [8]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
def clean_text(text):
    if isinstance(text , float): #Handle NaN or float values
        text=""

    text = text.lower() #convert to lowercase
    text = re.sub(r'\W', ' ', text) #remove special characters
    text = re.sub(r'\s+', ' ', text).strip() #remove extra spaces
    
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens] #lemmatization

    return ' '.join(tokens)

In [9]:
x 

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [10]:
x = x.apply(clean_text)

In [11]:
x

0        one reviewer mentioned watching 1 oz episode h...
1        wonderful little production br br filming tech...
2        thought wonderful way spend time hot summer we...
3        basically family little boy jake think zombie ...
4        petter mattei love time money visually stunnin...
                               ...                        
49995    thought movie right good job creative original...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    catholic taught parochial elementary school nu...
49998    going disagree previous comment side maltin on...
49999    one expects star trek movie high art fan expec...
Name: review, Length: 50000, dtype: object

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(x)

In [13]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2 , random_state=42)

In [14]:
y_train = y_train.replace({'positive':1 , 'negative':0})
y_test = y_test.replace({'positive':1 , 'negative':0})

C:\Users\Shiva\AppData\Local\Temp\ipykernel_22112\3646051714.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.replace({'positive':1 , 'negative':0})
C:\Users\Shiva\AppData\Local\Temp\ipykernel_22112\3646051714.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_test = y_test.replace({'positive':1 , 'negative':0})


XGB

In [15]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score , recall_score , f1_score
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import xgboost as xgb

In [16]:
def xgb_csv(max_depth , learning_rate , n_estimators , gamma , min_child_weight, subsample , colsample_bytree):
    params = {
        'max_depth' : int(max_depth),
        'learning_rate' : learning_rate,
        'n_estimators' : int(n_estimators),
        'gamma' : gamma,
        'min_child_weight' : min_child_weight,
        'subsample' : subsample,
        'colsample_bytree' : colsample_bytree,
        'objective' : 'binary:logistic',
        'eval_metric' : 'logloss'

    } 

    cv_results = cross_val_score(
        xgb.XGBClassifier(**params),
        x_train , y_train,
        scoring='accuracy',
        cv=5
    )

    return cv_results.mean()

In [17]:
pbounds = {
       'max_depth': (3, 10),
    'learning_rate': (0.01, 0.05),
    'n_estimators': (50, 500),
    'gamma': (0, 5),
    'min_child_weight': (1, 10),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1)
}

optimizer = BayesianOptimization(
    f=xgb_csv,
    pbounds = pbounds,
    random_state = 42

)

In [18]:
optimizer.maximize(
    init_points = 10,
    n_iter = 50
)

print(optimizer.max)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.8114    | 0.6873    | 4.754     | 0.03928   | 7.191     | 2.404     | 120.2     | 0.529     |
| 2         | 0.8266    | 0.9331    | 3.006     | 0.03832   | 3.144     | 9.729     | 424.6     | 0.6062    |
| 3         | 0.7972    | 0.5909    | 0.917     | 0.02217   | 6.673     | 4.888     | 181.1     | 0.8059    |
| 4         | 0.7897    | 0.5697    | 1.461     | 0.02465   | 6.192     | 8.067     | 139.9     | 0.7571    |
| 5         | 0.8338    | 0.7962    | 0.2323    | 0.0343    | 4.194     | 1.585     | 477.0     | 0.9828    |


KeyboardInterrupt: 

In [19]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score

# Your parameters
params = {
    'max_depth': 3,

    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'colsample_bytree': 0.7962, 
    'gamma': 0.2323,
    'learning_rate': 0.0343,
    'max_depth': 4,  # Changed from float to int
    'min_child_weight': 1.585,
    'n_estimators': 477,  # Changed from float to int
    'subsample': 0.9828
}

# Initialize model
model = xgb.XGBClassifier(**params)

# Define scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

# Initialize k-fold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation for each metric
precision_scores = cross_val_score(model, x_train, y_train, scoring=scoring['precision'], cv=skf)
recall_scores = cross_val_score(model, x_train, y_train, scoring=scoring['recall'], cv=skf)
f1_scores = cross_val_score(model, x_train, y_train, scoring=scoring['f1'], cv=skf)

# Print results
print(f"Mean Precision on training data: {np.mean(precision_scores):.4f}")
print(f"Mean Recall on training data: {np.mean(recall_scores):.4f}")
print(f"Mean F1-Score on training data: {np.mean(f1_scores):.4f}")

Mean Precision on training data: 0.8074
Mean Recall on training data: 0.8725
Mean F1-Score on training data: 0.8386


In [20]:
print(x_train.shape)
print(x_test.shape)

(40000, 92273)
(10000, 92273)


In [21]:
model.fit(x_train , y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7962, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.2323, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.0343, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=1.585, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=477,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [22]:
y_pred = model.predict(x_test)

In [23]:
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [24]:
# Calculate metrics on test set
print("\nTest Set Results:")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Test Set Results:

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.80      0.83      4961
           1       0.82      0.88      0.85      5039

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



In [25]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Confusion Matrix:
[[3964  997]
 [ 608 4431]]


In [26]:
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)


In [27]:
print("\nTest Set Metrics Summary:")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-Score: {test_f1:.4f}")



Test Set Metrics Summary:
Precision: 0.8163
Recall: 0.8793
F1-Score: 0.8467
